## Введение в обработку естественного языка
### Урок 3. Embedding word2vec fasttext

На основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы

In [2]:
import pandas as pd
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from gensim.models import Word2Vec, FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import tqdm
from inc.constants import INPUT_PATH
import os

In [3]:
df = pd.read_csv(os.path.join(INPUT_PATH,'lesson-3','lenta-ru-news.csv.gz'))
df.head()

,url,title,text,topic,tags
0,https://lenta.ru/news/2018/12/14/cancer/,Названы регионы России с самой высокой смертно...,Вице-премьер по социальным вопросам Татьяна Го...,Россия,Общество
1,https://lenta.ru/news/2018/12/15/doping/,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт,Зимние виды
2,https://lenta.ru/news/2018/12/15/disneyland/,Обнаружено самое счастливое место на планете,Сотрудники социальной сети Instagram проанализ...,Путешествия,Мир
3,https://lenta.ru/news/2018/12/15/usa25/,В США раскрыли сумму расходов на расследование...,С начала расследования российского вмешательст...,Мир,Политика
4,https://lenta.ru/news/2018/12/15/integrity/,Хакеры рассказали о планах Великобритании зами...,Хакерская группировка Anonymous опубликовала н...,Мир,Общество


In [19]:
txt_list = df.text.to_list()

In [4]:
def preprocess_txt(line, morpher):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [10]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(string.punctuation)
c = 0

In [20]:
for txt in txt_list:
    spls = preprocess_txt(txt, morpher)
    sentences.append(spls)
    c += 1
    if c > 1000:
        break

In [22]:
sentences = [i for i in sentences if len(i) > 2]
sentences[0][:10]

['вицепремьер',
 'социальный',
 'вопрос',
 'татьяна',
 'голиков',
 'рассказать',
 'регион',
 'россия',
 'зафиксировать',
 'высокий']

In [24]:
modelW2V = Word2Vec(sentences=sentences, vector_size=300, window=5, min_count=2)

In [25]:
w2v_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for question in sentences:
    n_w2v = 0
    vector_w2v = np.zeros(300)
    for word in question:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    w2v_index.add_item(counter, vector_w2v)
        
    counter += 1
    
    if counter > 1000:
        break

w2v_index.build(10)

True

In [42]:
def get_response(question, morpher, index, model, index_map, num_question):
    question = preprocess_txt(question, morpher)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, num_question, )
    return [txt_list[i] for i in answers]

In [43]:
TEXT = "Международный союз"
get_response(TEXT, morpher, w2v_index, modelW2V, index_map, 1)

['Комментатор Дмитрий Губерниев сообщил, что иностранные федерации настаивают на том, чтобы Международный союз биатлонистов (IBU) отстранил российских биатлонистов от участия в Кубке мира. Его слова приводит «Матч ТВ». По словам журналиста, представители Чехии, Канады и США стараются поднять вопрос об отстранении Алексея Волкова, Антона Шипулина, Александра Логинова, Евгения Гараничева и Ирины Старых от международных соревнований. Губерниев выразил мнение, что временная дисквалификация чаще всего чревата более серьезными последствиями. Вечером 12 декабря в отель сборной России в австрийском Хохфильцене прибыла полиция. Как написал биатлонист Александр Логинов, их обвинили в махинациях с переливанием крови. В Австрии прием допинга является уголовным преступлением. Максимальное наказание за его употребление — три года тюрьмы. IBU принял решение не применять санкции к российским биатлонистам, заподозренным в допинговых махинациях. Все они продолжают выступление на этапах Кубка мира.']

In [44]:
TEXT = "Правоохранительные органы"
get_response(TEXT, morpher, w2v_index, modelW2V, index_map, 1)

['По меньшей мере двое сотрудников правоохранительных органов ранены в результате взрыва в центре Назрани, сообщил «Ленте.ру» источник в правоохранительных органах. «Около 18:30 в центре города неизвестный бросил гранату в полицейский патруль. Оба сотрудника получили ранения и доставлены в реанимацию в тяжелом состоянии. Нападавшему удалось скрыться», — рассказал собеседник редакции. Как сообщил «Ленте.ру» другой источник в правоохранительных органах, инцидент произошел неподалеку от рынка «Фабрика» на улице Картоева. По предварительным данным, полицейские попытались проверить документы у двух подозрительных граждан, но те попытались скрыться. Сейчас на месте происшествия работает следственно-оперативная группа. Официальных комментариев «Ленте.ру» получить пока не удалось.']